In [1]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import csv
from datetime import datetime

# Librerías MicaSense (OBLIGATORIAS)
import micasense.capture as capture
import micasense.imageutils as imageutils
import micasense.plotutils as plotutils

# ============================================================
# CONFIGURACIÓN
# ============================================================
# Ruta de entrada (Windows)
base_path = r"C:\Users\win10\imagenes_articulo\Marzo2022"
output_base = r"C:\Users\win10\imagenes_articulo\resultados_multiespectrales"

# Crear carpeta de salida
os.makedirs(output_base, exist_ok=True)

# Archivo CSV de salida
csv_output = os.path.join(output_base, 'estadisticos_indices.csv')

print("="*80)
print("PROCESAMIENTO AUTOMÁTICO - IMÁGENES MULTIESPECTRALES MICASENSE")
print("="*80)
print(f"\nRuta entrada: {base_path}")
print(f"Ruta salida: {output_base}\n")

# ============================================================
# INICIALIZAR CSV
# ============================================================
fieldnames = [
    'nombre_imagen', 'fecha_proceso', 'num_bandas',
    # NDVI
    'max_NDVI', 'min_NDVI', 'mean_NDVI', 'std_NDVI', 'var_NDVI',
    # NDRE
    'max_NDRE', 'min_NDRE', 'mean_NDRE', 'std_NDRE', 'var_NDRE',
    # GNDVI
    'max_GNDVI', 'min_GNDVI', 'mean_GNDVI', 'std_GNDVI', 'var_GNDVI',
    # CVI
    'max_CVI', 'min_CVI', 'mean_CVI', 'std_CVI', 'var_CVI',
    # OSAVI
    'max_OSAVI', 'min_OSAVI', 'mean_OSAVI', 'std_OSAVI', 'var_OSAVI',
    # SCCCI
    'max_SCCCI', 'min_SCCCI', 'mean_SCCCI', 'std_SCCCI', 'var_SCCCI',
    # SAVI
    'max_SAVI', 'min_SAVI', 'mean_SAVI', 'std_SAVI', 'var_SAVI',
    # MACI
    'max_MACI', 'min_MACI', 'mean_MACI', 'std_MACI', 'var_MACI',
    # VARI
    'max_VARI', 'min_VARI', 'mean_VARI', 'std_VARI', 'var_VARI',
    # TCARI
    'max_TCARI', 'min_TCARI', 'mean_TCARI', 'std_TCARI', 'var_TCARI',
    # IPVI
    'max_IPVI', 'min_IPVI', 'mean_IPVI', 'std_IPVI', 'var_IPVI',
    # MSAVI
    'max_MSAVI', 'min_MSAVI', 'mean_MSAVI', 'std_MSAVI', 'var_MSAVI',
    # EVI
    'max_EVI', 'min_EVI', 'mean_EVI', 'std_EVI', 'var_EVI',
    # ARVI
    'max_ARVI', 'min_ARVI', 'mean_ARVI', 'std_ARVI', 'var_ARVI',
    # SIPI
    'max_SIPI', 'min_SIPI', 'mean_SIPI', 'std_SIPI', 'var_SIPI',
    # MTVI2
    'max_MTVI2', 'min_MTVI2', 'mean_MTVI2', 'std_MTVI2', 'var_MTVI2',
    # RDVI
    'max_RDVI', 'min_RDVI', 'mean_RDVI', 'std_RDVI', 'var_RDVI'
]

# Crear archivo CSV
with open(csv_output, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

print(f"✓ Archivo CSV creado: {csv_output}\n")

# ============================================================
# FUNCIÓN: AGRUPAR IMÁGENES POR CAPTURA
# ============================================================
def get_image_groups(folder_path):
    """Agrupa imágenes por nombre base"""
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))
    tif_files.extend(glob.glob(os.path.join(folder_path, "*.TIF")))
    
    groups = {}
    for file_path in tif_files:
        filename = os.path.basename(file_path)
        # Extraer nombre base: IMG_0683_1.tif -> IMG_0683
        parts = filename.rsplit('_', 1)
        if len(parts) == 2:
            base_name = parts[0]
            if base_name not in groups:
                groups[base_name] = []
            groups[base_name].append(file_path)
    
    # Ordenar archivos
    for base_name in groups:
        groups[base_name].sort()
    
    return groups

# ============================================================
# FUNCIÓN: PROCESAR CAPTURA MULTIESPECTRAL
# ============================================================
def process_multispectral_capture(base_name, image_files, output_path):
    """Procesa una captura completa con 17 índices de vegetación"""
    
    print(f"\n{'='*80}")
    print(f"PROCESANDO: {base_name}")
    print(f"Archivos: {len(image_files)}")
    print(f"{'='*80}\n")
    
    try:
        # ============================================================
        # 1. CARGAR CAPTURA CON MICASENSE
        # ============================================================
        print("--- Cargando captura ---")
        panelNames = glob.glob(os.path.join(base_path, 'panel_*.tif'))
        
        cap = capture.Capture.from_filelist(image_files)
        print(f"  ✓ Captura cargada: {cap.num_bands} bandas")
        
        # Verificar panel de calibración
        panelCap = None
        panel_reflectance_by_band = None
        
        if panelNames:
            try:
                panelCap = capture.Capture.from_filelist(panelNames)
                if panelCap.panel_albedo() is not None:
                    panel_reflectance_by_band = panelCap.panel_albedo()
                else:
                    panel_reflectance_by_band = [0.67, 0.69, 0.68, 0.61, 0.67]
                print(f"  ✓ Panel de calibración detectado")
            except:
                print(f"  ⚠ No se pudo cargar panel, usando valores por defecto")
                panel_reflectance_by_band = [0.67, 0.69, 0.68, 0.61, 0.67]
        
        # ============================================================
        # 2. ALINEACIÓN DE IMÁGENES
        # ============================================================
        print("\n--- Alineando imágenes ---")
        
        match_index = 4  # NIR
        max_alignment_iterations = 20
        warp_mode = cv2.MOTION_HOMOGRAPHY
        pyramid_levels = 3
        
        warp_matrices, alignment_pairs = imageutils.align_capture(
            cap,
            ref_index=match_index,
            max_iterations=max_alignment_iterations,
            warp_mode=warp_mode,
            pyramid_levels=pyramid_levels
        )
        
        print(f"  ✓ Imágenes alineadas correctamente")
        
        # ============================================================
        # 3. CONVERSIÓN A REFLECTANCIA O RADIANCIA
        # ============================================================
        print("\n--- Conversión radiométrica ---")
        
        cropped_dimensions, edges = imageutils.find_crop_bounds(
            cap, warp_matrices, warp_mode=warp_mode
        )
        
        # Determinar tipo de imagen
        if panelCap is not None and panel_reflectance_by_band is not None:
            panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)
            img_type = "reflectance"
            print(f"  ✓ Usando REFLECTANCIA (panel calibrado)")
        elif cap.dls_present():
            panel_irradiance = cap.dls_irradiance()
            img_type = "reflectance"
            print(f"  ✓ Usando REFLECTANCIA (DLS)")
        else:
            panel_irradiance = None
            img_type = "radiance"
            print(f"  ✓ Usando RADIANCIA")
        
        # Obtener imagen alineada
        im_aligned = imageutils.aligned_capture(
            cap, warp_matrices, warp_mode, cropped_dimensions,
            match_index, img_type=img_type
        )
        
        print(f"  • Dimensiones imagen alineada: {im_aligned.shape}")
        
        # ============================================================
        # 4. IDENTIFICAR BANDAS
        # ============================================================
        band_names_lower = [name.lower() for name in cap.band_names()]
        
        blue_band = band_names_lower.index('blue')
        green_band = band_names_lower.index('green')
        red_band = band_names_lower.index('red')
        rededge_band = band_names_lower.index('red edge')
        nir_band = band_names_lower.index('nir')
        
        print(f"  • Bandas: Blue={blue_band}, Green={green_band}, Red={red_band}, RedEdge={rededge_band}, NIR={nir_band}")
        
        # ============================================================
        # 5. CÁLCULO DE 17 ÍNDICES DE VEGETACIÓN
        # ============================================================
        print("\n--- Calculando 17 índices de vegetación ---")
        
        np.seterr(divide='ignore', invalid='ignore')
        
        # Extraer bandas
        blue = im_aligned[:, :, blue_band]
        green = im_aligned[:, :, green_band]
        red = im_aligned[:, :, red_band]
        rededge = im_aligned[:, :, rededge_band]
        nir = im_aligned[:, :, nir_band]
        
        # 1. NDVI (Normalized Difference Vegetation Index)
        ndvi = (nir - red) / (nir + red)
        print(f"  1. NDVI   ✓")
        
        # 2. NDRE (Normalized Difference Red Edge)
        ndre = (nir - rededge) / (nir + rededge)
        print(f"  2. NDRE   ✓")
        
        # 3. GNDVI (Green Normalized Difference Vegetation Index)
        gndvi = (nir - green) / (nir + green)
        print(f"  3. GNDVI  ✓")
        
        # 4. CVI (Chlorophyll Vegetation Index)
        cvi = (nir / green) * (red / green)
        print(f"  4. CVI    ✓")
        
        # 5. OSAVI (Optimized Soil Adjusted Vegetation Index)
        osavi = (nir - rededge) / (nir + rededge + 0.16)
        print(f"  5. OSAVI  ✓")
        
        # 6. SCCCI (Simplified Canopy Chlorophyll Content Index)
        sccci = ndre / ndvi
        print(f"  6. SCCCI  ✓")
        
        # 7. SAVI (Soil Adjusted Vegetation Index)
        L = 0.5
        savi = ((nir - red) / (nir + red + L)) * (1 + L)
        print(f"  7. SAVI   ✓")
        
        # 8. MACI (Modified Anthocyanin Content Index)
        maci = nir / green
        print(f"  8. MACI   ✓")
        
        # 9. VARI (Visible Atmospherically Resistant Index)
        vari = (green - red) / (green + red - blue)
        print(f"  9. VARI   ✓")
        
        # 10. TCARI (Transformed Chlorophyll Absorption Ratio Index)
        tcari = 3 * ((rededge - red) - 0.2 * (rededge - green) * (rededge / red))
        print(f"  10. TCARI ✓")
        
        # 11. IPVI (Infrared Percentage Vegetation Index)
        ipvi = nir / (nir + red)
        print(f"  11. IPVI  ✓")
        
        # 12. MSAVI (Modified Soil Adjusted Vegetation Index)
        msavi = (2 * nir + 1 - np.sqrt((2 * nir + 1)**2 - 8 * (nir - red))) / 2
        print(f"  12. MSAVI ✓")
        
        # 13. EVI (Enhanced Vegetation Index)
        G = 2.5
        C1 = 6.0
        C2 = 7.5
        L_evi = 1.0
        evi = G * ((nir - red) / (nir + C1 * red - C2 * blue + L_evi))
        print(f"  13. EVI   ✓")
        
        # 14. ARVI (Atmospherically Resistant Vegetation Index)
        arvi = (nir - (2 * red - blue)) / (nir + (2 * red - blue))
        print(f"  14. ARVI  ✓")
        
        # 15. SIPI (Structure Insensitive Pigment Index)
        sipi = (nir - blue) / (nir - red)
        print(f"  15. SIPI  ✓")
        
        # 16. MTVI2 (Modified Triangular Vegetation Index 2)
        mtvi2 = (1.5 * (1.2 * (nir - green) - 2.5 * (red - green))) / np.sqrt((2 * nir + 1)**2 - (6 * nir - 5 * np.sqrt(red)) - 0.5)
        print(f"  16. MTVI2 ✓")
        
        # 17. RDVI (Renormalized Difference Vegetation Index)
        rdvi = (nir - red) / np.sqrt(nir + red)
        print(f"  17. RDVI  ✓")
        
        # ============================================================
        # 6. CALCULAR ESTADÍSTICOS
        # ============================================================
        print("\n--- Calculando estadísticos ---")
        
        indices_dict = {
            'NDVI': ndvi, 'NDRE': ndre, 'GNDVI': gndvi, 'CVI': cvi,
            'OSAVI': osavi, 'SCCCI': sccci, 'SAVI': savi, 'MACI': maci,
            'VARI': vari, 'TCARI': tcari, 'IPVI': ipvi, 'MSAVI': msavi,
            'EVI': evi, 'ARVI': arvi, 'SIPI': sipi, 'MTVI2': mtvi2, 'RDVI': rdvi
        }
        
        stats_dict = {
            'nombre_imagen': base_name,
            'fecha_proceso': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'num_bandas': cap.num_bands
        }
        
        # Calcular estadísticos para cada índice
        for idx_name, idx_data in indices_dict.items():
            # Filtrar valores inválidos
            valid_data = idx_data[np.isfinite(idx_data)]
            
            if len(valid_data) > 0:
                stats_dict[f'max_{idx_name}'] = float(np.max(valid_data))
                stats_dict[f'min_{idx_name}'] = float(np.min(valid_data))
                stats_dict[f'mean_{idx_name}'] = float(np.mean(valid_data))
                stats_dict[f'std_{idx_name}'] = float(np.std(valid_data))
                stats_dict[f'var_{idx_name}'] = float(np.var(valid_data))
                print(f"  ✓ {idx_name}: min={stats_dict[f'min_{idx_name}']:.4f}, max={stats_dict[f'max_{idx_name}']:.4f}")
            else:
                stats_dict[f'max_{idx_name}'] = np.nan
                stats_dict[f'min_{idx_name}'] = np.nan
                stats_dict[f'mean_{idx_name}'] = np.nan
                stats_dict[f'std_{idx_name}'] = np.nan
                stats_dict[f'var_{idx_name}'] = np.nan
                print(f"  ⚠ {idx_name}: sin datos válidos")
        
        # ============================================================
        # 7. GUARDAR RESULTADOS
        # ============================================================
        print("\n--- Guardando resultados ---")
        
        # Crear carpeta para esta captura
        capture_folder = os.path.join(output_path, base_name)
        os.makedirs(capture_folder, exist_ok=True)
        
        # Guardar en CSV
        with open(csv_output, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(stats_dict)
        print(f"  ✓ Estadísticos guardados en CSV")
        
        # Guardar índices como imágenes PNG
        for idx_name, idx_data in indices_dict.items():
            valid_mask = np.isfinite(idx_data)
            if np.any(valid_mask):
                idx_min = np.min(idx_data[valid_mask])
                idx_max = np.max(idx_data[valid_mask])
                
                if idx_max > idx_min:
                    idx_normalized = np.zeros_like(idx_data, dtype=np.uint8)
                    idx_normalized[valid_mask] = ((idx_data[valid_mask] - idx_min) / (idx_max - idx_min) * 255).astype(np.uint8)
                    
                    img_path = os.path.join(capture_folder, f'{base_name}_{idx_name}.png')
                    cv2.imwrite(img_path, idx_normalized)
        
        print(f"  ✓ 17 imágenes de índices guardadas en: {capture_folder}")
        
        print(f"\n✅ COMPLETADO: {base_name}")
        
        return {'nombre': base_name, 'estado': 'Exitoso', 'indices': 17}
        
    except AttributeError as e:
        print(f"\n❌ ERROR de atributo en {base_name}: {str(e)}")
        print(f"   Posible problema con metadatos de la imagen")
        return {'nombre': base_name, 'estado': f'Error: Atributo faltante - {str(e)}', 'indices': 0}
    
    except Exception as e:
        print(f"\n❌ ERROR en {base_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        return {'nombre': base_name, 'estado': f'Error: {str(e)}', 'indices': 0}

# ============================================================
# EJECUCIÓN PRINCIPAL
# ============================================================

print("\n--- Escaneando carpeta ---")
image_groups = get_image_groups(base_path)

print(f"\n{'='*80}")
print(f"CAPTURAS DETECTADAS: {len(image_groups)}")
print(f"{'='*80}\n")

for base_name, files in sorted(image_groups.items()):
    print(f"  • {base_name}: {len(files)} archivos")

if len(image_groups) == 0:
    print("\n⚠ No se encontraron imágenes .tif en la carpeta")
    print(f"   Ruta: {base_path}")
else:
    # Procesar cada grupo
    results = []
    
    for i, (base_name, image_files) in enumerate(sorted(image_groups.items()), 1):
        print(f"\n{'#'*80}")
        print(f"# CAPTURA {i}/{len(image_groups)}")
        print(f"{'#'*80}")
        
        result = process_multispectral_capture(base_name, image_files, output_base)
        results.append(result)
    
    # Resumen final
    print(f"\n\n{'='*80}")
    print("RESUMEN FINAL")
    print(f"{'='*80}\n")
    
    df_results = pd.DataFrame(results)
    print(df_results.to_string(index=False))
    
    exitosos = sum(1 for r in results if r['estado'] == 'Exitoso')
    
    print(f"\n{'='*80}")
    print(f"Total procesadas: {len(results)}")
    print(f"Exitosas: {exitosos}")
    print(f"Con errores: {len(results) - exitosos}")
    print(f"\nResultados en: {output_base}")
    print(f"CSV completo: {csv_output}")
    print(f"{'='*80}\n")

PROCESAMIENTO AUTOMÁTICO - IMÁGENES MULTIESPECTRALES MICASENSE

Ruta entrada: C:\Users\win10\imagenes_articulo\Marzo2022
Ruta salida: C:\Users\win10\imagenes_articulo\resultados_multiespectrales

✓ Archivo CSV creado: C:\Users\win10\imagenes_articulo\resultados_multiespectrales\estadisticos_indices.csv


--- Escaneando carpeta ---

CAPTURAS DETECTADAS: 35

  • IMG_0305: 10 archivos
  • IMG_0306: 10 archivos
  • IMG_0373: 10 archivos
  • IMG_0374: 10 archivos
  • IMG_0375: 10 archivos
  • IMG_0376: 10 archivos
  • IMG_0377: 10 archivos
  • IMG_0378: 10 archivos
  • IMG_0379: 10 archivos
  • IMG_0380: 10 archivos
  • IMG_0381: 10 archivos
  • IMG_0382: 10 archivos
  • IMG_0399: 10 archivos
  • IMG_0416: 10 archivos
  • IMG_0417: 10 archivos
  • IMG_0418: 10 archivos
  • IMG_0419: 10 archivos
  • IMG_0420: 10 archivos
  • IMG_0421: 10 archivos
  • IMG_0422: 10 archivos
  • IMG_0423: 10 archivos
  • IMG_0424: 10 archivos
  • IMG_0425: 10 archivos
  • IMG_0426: 10 archivos
  • IMG_0427: 10 

Finished aligning band 7
Finished aligning band 6
Finished aligning band 3
Finished aligning band 1
Finished aligning band 8
  ✓ Imágenes alineadas correctamente

--- Conversión radiométrica ---
  ✓ Usando RADIANCIA
  • Dimensiones imagen alineada: (925, 1239, 10)
  • Bandas: Blue=0, Green=2, Red=4, RedEdge=8, NIR=6

--- Calculando 17 índices de vegetación ---
  1. NDVI   ✓
  2. NDRE   ✓
  3. GNDVI  ✓
  4. CVI    ✓
  5. OSAVI  ✓
  6. SCCCI  ✓
  7. SAVI   ✓
  8. MACI   ✓
  9. VARI   ✓
  10. TCARI ✓
  11. IPVI  ✓
  12. MSAVI ✓
  13. EVI   ✓
  14. ARVI  ✓
  15. SIPI  ✓
  16. MTVI2 ✓
  17. RDVI  ✓

--- Calculando estadísticos ---
  ✓ NDVI: min=-0.0773, max=0.9642
  ✓ NDRE: min=-0.1373, max=0.7887
  ✓ GNDVI: min=-0.1221, max=0.8475
  ✓ CVI: min=0.2455, max=12.9475
  ✓ OSAVI: min=-0.0280, max=0.1352
  ✓ SCCCI: min=-28675.7969, max=1543.8680
  ✓ SAVI: min=-0.0094, max=0.1273
  ✓ MACI: min=0.7824, max=12.1112
  ✓ VARI: min=-0.3695, max=1.3111
  ✓ TCARI: min=-0.0600, max=0.0756
  ✓ IPVI: min=0.

multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\win10\anaconda3\envs\micasense2\lib\multiprocessing\pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "C:\Users\win10\imagenes_articulo\micasense\imageutils.py", line 163, in align
    cc, warp_matrix = cv2.findTransformECC(grad1, grad2, warp_matrix, warp_mode, criteria, inputMask=None, gaussFiltSize=1)
cv2.error: OpenCV(4.6.0) D:\bld\libopencv_1666532770404\work\modules\video\src\ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'cv::findTransformECC'

"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\win10\AppData\Local\Temp\ipykernel_43864\1575339927.py", line 158, in process_multispectral_capture
    pyramid_levels=pyramid_levels
  File "C:\

  ✓ Captura cargada: 10 bandas

--- Alineando imágenes ---
Finished aligning band 4

❌ ERROR en IMG_0377: OpenCV(4.6.0) D:\bld\libopencv_1666532770404\work\modules\video\src\ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'cv::findTransformECC'


################################################################################
# CAPTURA 8/35
################################################################################

PROCESANDO: IMG_0378
Archivos: 10

--- Cargando captura ---


multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\win10\anaconda3\envs\micasense2\lib\multiprocessing\pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "C:\Users\win10\imagenes_articulo\micasense\imageutils.py", line 163, in align
    cc, warp_matrix = cv2.findTransformECC(grad1, grad2, warp_matrix, warp_mode, criteria, inputMask=None, gaussFiltSize=1)
cv2.error: OpenCV(4.6.0) D:\bld\libopencv_1666532770404\work\modules\video\src\ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'cv::findTransformECC'

"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\win10\AppData\Local\Temp\ipykernel_43864\1575339927.py", line 158, in process_multispectral_capture
    pyramid_levels=pyramid_levels
  File "C:\

...

multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\win10\anaconda3\envs\micasense2\lib\multiprocessing\pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "C:\Users\win10\imagenes_articulo\micasense\imageutils.py", line 163, in align
    cc, warp_matrix = cv2.findTransformECC(grad1, grad2, warp_matrix, warp_mode, criteria, inputMask=None, gaussFiltSize=1)
cv2.error: OpenCV(4.6.0) D:\bld\libopencv_1666532770404\work\modules\video\src\ecc.cpp:589: error: (-7:Iterations do not converge) The algorithm stopped before its convergence. The correlation is going to be minimized. Images may be uncorrelated or non-overlapped in function 'cv::findTransformECC'

"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\win10\AppData\Local\Temp\ipykernel_43864\1575339927.py", line 158, in process_multispectral_capture
    pyramid_levels=pyramid_levels
  File "C:\

  ✓ Captura cargada: 10 bandas

--- Alineando imágenes ---
Finished aligning band 4

❌ ERROR en IMG_0439: OpenCV(4.6.0) D:\bld\libopencv_1666532770404\work\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 1228800 bytes in function 'cv::OutOfMemoryError'


################################################################################
# CAPTURA 35/35
################################################################################

PROCESANDO: IMG_0440
Archivos: 10

--- Cargando captura ---


multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\win10\anaconda3\envs\micasense2\lib\multiprocessing\pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "C:\Users\win10\imagenes_articulo\micasense\imageutils.py", line 163, in align
    cc, warp_matrix = cv2.findTransformECC(grad1, grad2, warp_matrix, warp_mode, criteria, inputMask=None, gaussFiltSize=1)
cv2.error: OpenCV(4.6.0) D:\bld\libopencv_1666532770404\work\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 1228800 bytes in function 'cv::OutOfMemoryError'

"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\win10\AppData\Local\Temp\ipykernel_43864\1575339927.py", line 158, in process_multispectral_capture
    pyramid_levels=pyramid_levels
  File "C:\Users\win10\imagenes_articulo\micasense\imageutils.py", line 233, in align_capture
    for _,mat in enumera

  ✓ Captura cargada: 10 bandas

--- Alineando imágenes ---

❌ ERROR en IMG_0440: Unable to allocate 9.38 MiB for an array with shape (1280, 960) and data type float64


RESUMEN FINAL

  nombre                                                                                                                                                                                                                                                                                                   estado  indices
IMG_0305                                                                                                                                                                                                                                                                                                  Exitoso       17
IMG_0306                                                                                                                                                                                  

Traceback (most recent call last):
  File "C:\Users\win10\AppData\Local\Temp\ipykernel_43864\1575339927.py", line 158, in process_multispectral_capture
    pyramid_levels=pyramid_levels
  File "C:\Users\win10\imagenes_articulo\micasense\imageutils.py", line 217, in align_capture
    'match_image':img.undistorted(img.radiance()).astype('float32'),
  File "C:\Users\win10\imagenes_articulo\micasense\image.py", line 309, in radiance
    L = V * R * (image_raw - self.black_level)
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 9.38 MiB for an array with shape (1280, 960) and data type float64
